## Sinusoids Dataset
This script computes an example dataset made of sinusoids of length 1 sec at frequency sampling of 44100 Hz.

In [ ]:
import numpy as np
import librosa # write the output for check purpose
from datetime import datetime

In [ ]:
# calculate a quasi-mel spaced array of frequencies
freqs = np.arange(50,100,5)
freqs = np.concatenate((freqs,np.arange(100,500,10)))
freqs = np.concatenate((freqs,np.arange(510,1000,20)))
freqs = np.concatenate((freqs,np.arange(1000,2000,40)))
freqs = np.concatenate((freqs,np.arange(2000,4000,80)))
print('no of selected frequencies:', freqs.shape)

In [ ]:
# define a function that computes a sine wave
def sinusoid(freq, fs, dur):
    y = (np.sin(2*np.pi*np.arange(fs*dur)*freq/fs)).astype(np.float32)
         
    return y

In [ ]:
# variables initialisation
fs = 44100
dur = 1
dataset = np.zeros([44100]).astype(np.float32)

In [ ]:
# For a set containing one sinusouids per example  
print('Computation started at ', datetime.now().time())

for freq in freqs:
    # initialise y to be empty
    y = 0
    
    # calculate the sinusoid 
    y = sinusoid(freq=freq,sr=sr,dur=dur,amp=1)
    # stack the sinusoid in the numpy array
    dataset = np.vstack((dataset,y))

In [ ]:
# For a set containing more sinusouids per example  
print('Computation started at ', datetime.now().time())

# compute up to 10 harmonics for each frequency in the list and sum them up
for freq in freqs:
    # initialise y to be empty
    y = np.zeros([44100,]).astype(np.float32)
    for n in range(10):
        # calculate frequency if < 20kHz
        x = freq*(n+1)
        #print('freq:', x)
        if x < 20000:
            # calculate the sinusoid and add it to the previous one(s)
            y += sinusoid(freq=x,fs=fs,dur=dur)
            # apply normalisation
            y /= np.max(abs(y))
            #print(np.max(abs(y)))
                
            # stack the sinusoid in the numpy array
            dataset = np.vstack((dataset,y))
        else: 
            pass
        


In [ ]:
np.savez_compressed(file='sine_dataset', sines=dataset)
print('Saved and successfully finished at ', datetime.now().time())

## Log spectra dataset

In [ ]:
# define function for computing log magnitude spectra
def compute_spectra(x, fft_size, hop_size):

    # get options for computing spectra
    no_fft = fft_size                      # fft length
    hop = hop_size                         # hop size

    # STFT for the vector, 50% overlap
    # (S is a complex matrix with the magnitude and phase of the fourier transform)
    S = librosa.core.stft(y=x,n_fft=no_fft,hop_length=hop_size) 
    # Take the magnitude with abs (call it Z) and return log(1 + Z) as the spectrogram
    S = np.log1p(np.abs(S)) 
        
    # get spectrogram matrix dimensions (to be used in the convnet)
    dims = S.shape
    # print(dims)

    return S, dims

In [ ]:
# general parameter initialisation
sr = 44100                                              # sampling rate
fft_size = 1024                                         # size of FFT
hop_size = 512                                          # hop size (overlap)
duration = 1                                            # segment duration (sec)
no_fft = 88                                             # frames in spectrogram
res = fft_size/2+1

# load the dataset structure
path = 'sine_dataset.npz'
data = np.load(path, 'rb')
sines = data['sines']                                 # get the id for each recording (list)
# discard first
sines = sines[1:]
print(len(sines), ' sinusoids in the dataset')

# variables initialisation
dataset = np.empty([0, res, no_fft])                    # empty 3-d array
norm_coeffs = np.empty(0,).astype(np.float32)           # for normalisation factors
mean_coeffs = np.empty(0,).astype(np.float32)           # for standardisation factors

In [ ]:
# SPECTROGRAM COMPUTATION
print('Computation started at ', datetime.now().time())
for sine in sines:
    
    # zero-pad to get 88 timesteps in spectrogram
    x = np.hstack(tup=(sine,np.zeros(500,)))
    # hard coding for 128-by-128 spectrogram
    #x = sine[:65500]
    # compute the spectrogram for the sample
    [S, dims] = compute_spectra(
        x=x,
        fft_size=fft_size,
        hop_size=hop_size,
    )
    
    # normalise the spectrogram
    # reshape spectrogram
    S = np.reshape(S, newshape=[1, dims[0], dims[1]])
    dataset = np.concatenate((dataset, S), axis=0)
    #norm_coeffs = np.hstack(tup=(norm_coeffs,S_norm))
    #mean_coeffs = np.hstack(tup=(mean_coeffs,S_mean))
    #print('New shape:', dataset.shape)

In [ ]:
# DATASET STORAGE
# obs: maybe saving the norm factor for reconstruction test
# check for the dataset shape (expected: batch, mels, timesteps)
print('Dataset shape:',dataset.shape)
# save the dataset
np.savez_compressed(file='sineSpectra_dataset', 
                    sines=dataset.astype(np.float32) 
                   )
print('Saved and successfully finished at ', datetime.now().time())